In [28]:
import pandas as pd
import requests
from datetime import datetime
from geopy.distance import geodesic

# **Función carga de archivo NGS Cors local**

Lee archivo de manera local el registro de las estaciones existentes de la RED CORS NGS para obtener la lsita de estaciones y sus posiciones aproximadas.

In [27]:
def cargar_estaciones_local(ruta_csv="NOAACORSNetwork.csv"):
    df = pd.read_csv(ruta_csv, sep=",", encoding="utf-8-sig", on_bad_lines='warn')
    if 'x' in df.columns and 'y' in df.columns:
        df = df[df['x'].str.contains(",", na=False) & df['y'].str.contains(",", na=False)].copy()
        df['Longitude'] = df['x'].str.replace(",", ".", regex=False).astype(float)
        df['Latitude'] = df['y'].str.replace(",", ".", regex=False).astype(float)
    elif 'LATITUDE' in df.columns and 'LONGITUDE' in df.columns:
        df['Latitude'] = df['LATITUDE'].str.replace(",", ".", regex=False).astype(float)
        df['Longitude'] = df['LONGITUDE'].str.replace(",", ".", regex=False).astype(float)
    else:
        raise ValueError("No se encontraron columnas válidas de coordenadas.")
    return df

# **Función de obtener el día del año**
De acuerdo al formato de fecha convencional obtiene el año y el día del año para busqueda de archivos.

In [29]:
def fecha_a_doy(fecha_str):
    fecha = datetime.strptime(fecha_str, "%Y-%m-%d")
    anio = fecha.year
    doy = fecha.timetuple().tm_yday
    return anio, doy

#**Función de obtener estaciones cercanas**

Se obtienen estaciones más cercanas

In [30]:
def estaciones_mas_cercanas(df, lat_usuario, lon_usuario, n=4):
    punto_usuario = (lat_usuario, lon_usuario)
    df['Distancia_km'] = df.apply(lambda row: geodesic(punto_usuario, (row['Latitude'], row['Longitude'])).kilometers, axis=1)
    return df.sort_values('Distancia_km').head(n).copy()

# **Función nombrar archivo de acuerdo a nomenclatura**

Se nombra los archivos para construir el vinculo

In [31]:
def generar_nombre_archivo(siteid, anio, doy, tipo='obs'):
    siteid = siteid.lower()
    yy = str(anio)[-2:]
    doy_str = str(doy).zfill(3)
    if tipo == 'obs':
        return f"{siteid}{doy_str}0.{yy}o.gz"
    elif tipo == 'crx':
        return f"{siteid}{doy_str}0.{yy}d.gz"
    else:
        raise ValueError("Tipo inválido: usa 'obs' o 'crx'.")

# **Función de comprobación de disponibilidad del archivo**

Obtiene una lista de vinculos disponibles de la base de datos (hace una consulta), de esta manera compara los vinculos que se generaron y evalua disponibilidad de los datos.

In [32]:
def verificar_disponibilidad_rinex(df_cercanas, anio, doy, tipo='obs'):
    doy_str = str(doy).zfill(3)
    url_lista = f"https://noaa-cors-pds.s3.amazonaws.com/rinex/{anio}/{doy_str}/{anio}.{doy_str}.files.list"
    try:
        r = requests.get(url_lista)
        r.raise_for_status()
        contenido = r.text
    except Exception as e:
        print(f"❌ Error al obtener lista de archivos: {e}")
        df_cercanas['Disponible'] = "ERROR"
        df_cercanas['URL'] = None
        return df_cercanas

    disponibles = []
    urls = []

    for _, row in df_cercanas.iterrows():
        siteid = row['SITEID'].lower()
        nombre_archivo = generar_nombre_archivo(siteid, anio, doy, tipo)
        if nombre_archivo in contenido:
            disponibles.append("SI")
            url = f"https://noaa-cors-pds.s3.amazonaws.com/rinex/{anio}/{doy_str}/{siteid}/{nombre_archivo}"
            urls.append(url)
        else:
            disponibles.append("NO")
            urls.append(None)

    df_cercanas['Disponible'] = disponibles
    df_cercanas['URL'] = urls
    return df_cercanas


# **DESCARGA**

Descarga del vinculo según la selección del url seleccionado.

In [33]:
def descargar_archivo_rinex(url):
    try:
        nombre_archivo = url.split("/")[-1]
        r = requests.get(url)
        with open(nombre_archivo, "wb") as f:
            f.write(r.content)
    except Exception as e:
        print(f" Error al descargar: {e}")

# **SUGERENCIA DE FRONTEND**

In [35]:
df_estaciones = cargar_estaciones_local()

# 2. Pedir coordenadas del usuario
lat = float(input(" Ingrese su latitud (ej. 34.05): "))
lon = float(input(" Ingrese su longitud (ej. -118.25): "))

# 3. Pedir fecha
fecha = input(" Ingrese la fecha (YYYY-MM-DD): ")
anio, doy = fecha_a_doy(fecha)

# 4. Buscar estaciones cercanas y verificar archivos
df_cercanas = estaciones_mas_cercanas(df_estaciones, lat, lon)
df_cercanas = verificar_disponibilidad_rinex(df_cercanas, anio, doy, tipo='obs')

# 5. Mostrar tabla
print("\n🧭 Estaciones más cercanas:")
print(df_cercanas[['SITEID', 'Latitude', 'Longitude', 'Distancia_km', 'SAMPLING', 'AGENCY', 'STATUS', 'Disponible']].to_string(index=False))

# 6. Selección válida de estación disponible
while True:
    siteid = input("\n Ingrese el SITEID de una estación disponible (o escriba 'salir' para cancelar): ").strip().upper()

    if siteid.lower() == 'salir':
        print(" Operación cancelada por el usuario.")
        break

    fila_sel = df_cercanas[df_cercanas['SITEID'] == siteid]

    if fila_sel.empty:
        print(" El SITEID ingresado no está en la lista mostrada. Intente nuevamente.")
    elif fila_sel.iloc[0]['Disponible'] != "SI":
        print(f" El archivo de {siteid} para el {fecha} NO está disponible. Intente con otro.")
    else:
        url = fila_sel.iloc[0]['URL']
        descargar_archivo_rinex(url)
        print(f" ✔️ Archivo de {siteid} para el {fecha} descargado exitosamente.")
        break

 Ingrese su latitud (ej. 34.05): 50
 Ingrese su longitud (ej. -118.25): -90
📅 Ingrese la fecha (YYYY-MM-DD): 2023-12-01

🧭 Estaciones más cercanas:
SITEID  Latitude  Longitude  Distancia_km SAMPLING                                 AGENCY          STATUS Disponible
  MNAS    48.294    -92.970    287.964818        5 Minnesota Department of Transportation     Operational         SI
  MICP    47.467    -87.874    322.151255        1  Michigan Department of Transportation Non-Operational         SI
  MNBD    48.628    -94.067    332.697901        5 Minnesota Department of Transportation     Operational         SI
  MNVI    47.523    -92.561    333.611517        5 Minnesota Department of Transportation     Operational         SI

 Ingrese el SITEID de una estación disponible (o escriba 'salir' para cancelar): SRS1
 El SITEID ingresado no está en la lista mostrada. Intente nuevamente.

 Ingrese el SITEID de una estación disponible (o escriba 'salir' para cancelar): salir
 Operación cancelada 